In [ ]:
import cv2
import numpy as np

from skimage import data, exposure
from skimage.feature import blob_dog, blob_log, blob_doh
from skimage.color import rgb2gray

import matplotlib.pyplot as plt

def preprocess_img(img_path):
    image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    # Normalize image
    min_bit = np.min(image)
    max_bit = np.max(image)
    image = cv2.normalize(image*16, dst=None, alpha=min_bit*16, beta=max_bit*16, norm_type=cv2.NORM_MINMAX)
    return image

def print_img(preprocessed_img, title="Image"):    
    fig, ax = plt.subplots(figsize=(9, 3))
    ax.set_title(title)
    ax.imshow(preprocessed_img, cmap=plt.cm.gray)
    
bf_path = "R:/Users/Rahul/Documents/494/Data/88hr timelapse/EGFP_1mm/BF/"
day_0 = "R:/Users/Rahul/Documents/494/Data/Day 0/1mm EGFP/"

sample_spheroid = "R:/Users/Rahul/Documents/494/Data/88hr timelapse/EGFP_1mm/BF/scan_Plate_R_p00_0_A02f00d4.TIF"
sample_sensor = "R:/Users/Rahul/Documents/494/Data/88hr timelapse/EGFP_1mm/Texas Red/scan_Plate_R_p00_0_A02f00d3.TIF"  

In [ ]:
sample_spheroid = "R:/Users/Rahul/Documents/494/Data/88hr timelapse/EGFP_1mm/BF/scan_Plate_R_p00_0_A02f00d4.TIF"
sample_sensor = "R:/Users/Rahul/Documents/494/Data/88hr timelapse/EGFP_1mm/Texas Red/scan_Plate_R_p00_0_A02f00d3.TIF" 

In [ ]:
sample_spheroid = "R:/Users/Rahul/Documents/494/Data/Day 0/500um Src/500mmSrc_Day0_Plate_R_p00_0_B01f00d4.TIF"
sample_sensor = "R:/Users/Rahul/Documents/494/Data/Day 0/500um Src/500mmSrc_Day0_Plate_R_p00_0_B01f00d3.TIF"

## 88hr timelapse - Src_500um - First Picture

In [ ]:
sample_spheroid = "R:/Users/Rahul/Documents/494/Data/88hr timelapse/Src_500um/BF/scan_Plate_R_p00_0_B01f00d4.TIF"
sample_sensor = "R:/Users/Rahul/Documents/494/Data/88hr timelapse/Src_500um/Texas Red/scan_Plate_R_p00_0_B01f00d3.TIF"

In [ ]:
isSensor = False

threshold = 100 if isSensor else 120

img = sample_sensor if isSensor else sample_spheroid
img = preprocess_img(img)

_, thresh = cv2.threshold(img, threshold, np.max(img), cv2.THRESH_BINARY)

In [ ]:
# Retrieval modes and contour approximation types found on OpenCV docs
raw_contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, 
                                       cv2.CHAIN_APPROX_NONE)

## Circle detection

In [ ]:
radius_range = (15, 100) if isSensor else (40, 500)

contours = []
for contour in raw_contours:
    # Most items in raw contours are lines or small shapes
    if cv2.contourArea(contour) < (np.pi * radius_range[0] ** 2):
        continue
    contours.append(contour)    

circle_coords = []
for contour in contours:
    (x, y), r = cv2.minEnclosingCircle(contour)
    if r > radius_range[1]:
        continue
    circle_coords.append((x, y, r))

In [ ]:
verbose = True

fig, ax = plt.subplots(figsize=(9, 3))

ax.set_title("MinEnclosingCircle")
ax.imshow(img, cmap=plt.cm.gray)

print("All units in um ({} pixels/um)\n".format(scale))
for idx, circle in enumerate(circle_coords):
    x, y, r = circle[0], circle[1], circle[2]
    c = plt.Circle((x, y), r, color="red", linewidth=2, fill=False)
    
    if verbose:
        ax.text(x + r, y, str(idx+1), color='red', fontsize=15)
        
        scale = 0.638
        x, y, r = x/scale, y/scale, r/scale
        print("Circle #{}:\n center (x,y): ({}, {})\n area: {}".format(idx+1,x,y,np.pi*r**2))
        
    ax.add_patch(c)
    
ax.set_axis_off()

In [ ]:
import xlsxwriter

In [ ]:
workbook = xlsxwriter.Workbook("C:/Users/Rahul Behal/Desktop/test2.xlsx")
worksheet = workbook.add_worksheet("Raw Data")

h1_format = workbook.add_format({
    'bold': 1,
    'underline': 1,
    'align': 'center',
    'valign': 'vcenter',
    'fg_color': '#FFD966'})
h2_format = workbook.add_format({
    'bold': 1,
    'underline': 1,
    'align': 'center',
    'valign': 'vcenter',
    'fg_color': '#F4B084'})
h3_format = workbook.add_format({
    'bold': 1,
    'align': 'center',
    'valign': 'vcenter',
    'fg_color': '#C6E0B4'})
id_format = workbook.add_format({
    'bold': 1,
    'align': 'left',
    'fg_color': '#8EA9DB'})
data_format = workbook.add_format({'align': 'right','fg_color': '#C6E0B4'})
data_format.set_num_format('0.00')

In [ ]:
data = {'p02': ({'1': ['365269.40712547314', '2488.244514106583', '1596.3949843260189', '340.9822039469656', '340.9822039469656', '0'], '2': ['223851.89996122388', '1380.0719449512637', '824.91797563798', '266.9349598660364', '266.9349598660364', '0']}, {'1a': ['2986.7974600324383', '2423.662430814067', '1853.3110902600902', '66.26275041634013', '57.39134905106595', 
'63.865631103515625'], '1b': ['12510.93456137555', '2479.1829937304074', '1698.1511130975705', '136.27348275020205', '116.89300297943402', '30.570865631103516'], '2a': ['908.2700111213213', '1431.3133502828664', '718.6528029486677', '36.642651572870236', '31.56008765241569', '7.639884948730469']}), 'p00': ({'1': ['411759.2528030942', '2456.5217308116185', '1589.6783428132346', '362.03182304167075', '362.03182304167075', '0'], '2': ['263809.2263241648', '1404.000691859326', '817.7392654658111', '289.7810980817741', '289.7810980817741', '0']}, {'1a': ['3426.1757408246467', '2411.6044477982955', '1866.6529162176723', '76.14037086223733', '57.293422543516726', '80.4987564086914'], '1b': ['14985.163505837763', '2475.3639914772725', '1688.5259263567789', '146.6280419998408', '130.12315038603302', '22.962120056152344'], '2a': ['1983.456782884504', '1429.697449304467', '725.7778921097423', '51.42788901971799', '49.10595514183881', '19.82471466064453']}), 'p01': ({'1': ['358753.45656505664', '2500.064096480701', '1604.2554131869613', '337.92716956063873', '337.92716956063873', '0'], '2': ['232930.15192237854', '1392.092618075284', '816.1655533650078', '272.2939039846199', '272.2939039846199', '0']}, {'1a': ['3974.5447164205184', '2420.5146239469045', '1863.6149343652037', '83.07676404994857', '60.914114351182896', '81.33804035186768'], '1b': ['14449.666105361177', '2475.0178704814853', '1694.2368271208854', '143.54736902123335', '128.16595956449598', '28.087814331054688'], '2a': ['1148.1521093521646', '1431.261020768025', '722.0631662207337', '44.396523024221196', '32.927638684694294', '27.459762573242188']})} 
baseId = 'p00'
spheroidIds = sorted(data[baseId][0].keys())
sensorIds = sorted(data[baseId][1].keys())

In [ ]:
sensorCount = {}
for spheroidId in spheroidIds:
    sensorList = []
    for sensorId in sensorIds:
        if int(spheroidId) == int(sensorId[0]):
            sensorList.append(sensorId)
    sensorCount[spheroidId] = sensorList

In [ ]:
headerRow = ["ID#","AREA","X","Y","MAJOR","MINOR","ADJ. ANGLE","ID#","AREA","X","Y","MAJOR","MINOR","ADJ. ANGLE"]     

In [ ]:
def writeRawDayData(startRow, startCol, dayId):
    rowNum, colNum = startRow, startCol
    
    # Convert spheroid and sensor data into arrays of strings for Excel write
    spheroidRows = []
    for id_ in spheroidIds:
        rowData = data[dayId][0].get(id_,['','','','','','',''])
        spheroidRows.append([str(id_)] + rowData)
    sensorRows = []
    for id_ in sensorIds:
        rowData = data[dayId][1].get(id_,['','','','','','',''])
        sensorRows.append([str(id_)] + rowData)       
    
    # Header Rows
    worksheet.merge_range(startRow, startCol, startRow, startCol + 13, "DAY{}".format(dayId.upper()), h1_format)
    rowNum = rowNum + 1
    worksheet.merge_range(rowNum, startCol, rowNum, startCol + 6, "SPHEROID", h2_format)
    worksheet.merge_range(rowNum, startCol + 7, rowNum, startCol + 13, "SENSOR", h2_format)
    rowNum = rowNum + 1
    for i in range(len(headerRow)):
        entry = headerRow[i]
        if i == 0 or i == 7:
            worksheet.write(rowNum, colNum, entry, id_format)
        else:
            worksheet.write(rowNum, colNum, entry, h3_format)
        colNum = colNum + 1
    rowNum = rowNum + 1
    colNum = startCol



    # Spheroid Rows
    data_start = rowNum, colNum

    for row in spheroidRows:
        for j in range(len(row)):
            entry = row[j]
            if j > 0:
                worksheet.write_number(rowNum, colNum, float(entry), data_format)
            else:
                worksheet.write_number(rowNum, colNum, float(entry), id_format)
            colNum = colNum + 1
        currRow = rowNum

        rowNum = rowNum + 1
        colNum = startCol   

        if row == spheroidRows[-1]:
            break
        while rowNum < currRow + len(sensorCount[row[0]]) + 1:
            for j in range(len(row)):
                if j > 0:
                    worksheet.write(rowNum, colNum, "", data_format)
                else:
                    worksheet.write(rowNum, colNum, "", id_format)
                colNum = colNum + 1     
            rowNum = rowNum + 1
            colNum = startCol



    # Sensor Rows
    rowNum, colNum = data_start
    colNum = colNum + 7

    for i in range(len(sensorRows)):
        row = sensorRows[i]
        for j in range(len(row)):
            entry = row[j]
            if j > 0:
                worksheet.write_number(rowNum, colNum, float(entry), data_format)
            else:
                worksheet.write(rowNum, colNum, entry, id_format)
            colNum = colNum + 1

        if i < len(sensorRows) - 1:
            if int(sensorRows[i+1][0][0]) != int(row[0][0]):
                rowNum = rowNum + 1  
                colNum = startCol + 7
                for j in range(len(row)):
                    if j > 0:
                        worksheet.write(rowNum, colNum, "", data_format)
                    else:
                        worksheet.write(rowNum, colNum, "", id_format)
                    colNum = colNum + 1     

        rowNum = rowNum + 1            
        colNum = startCol + 7

In [ ]:
# Start
dayIds = sorted(data.keys())
rowNum, colNum = 0, 0
for i in range(len(dayIds)):
    id_ = dayIds[i]
    writeRawDayData(rowNum, colNum+(15*i), id_)
worksheet.set_column(0, 15*len(dayIds), 10)    

In [ ]:
workbook.close()